<h2>Biopython (deel 2) en BioMart</h2>

Gebruik voor opdracht 1 de Modules Bio.AlignIO en Bio.Align.AlignInfo. Documentatie en voorbeelden kun je vinden op: http://biopython.org/DIST/docs/tutorial/Tutorial.html.

<b>Opdracht 1: Alignments</b>

Het bestand 'clustalw.aln' bevat een alignment in clustal format.

a) Zet het bestand om in fasta-format en sla dit op als 'fasta.aln'.<br>
b) Print de lengte van de alignment.<br>
c) Maak een Position-Specific Score Matrix van de alignment.<br>
d) Gebruik de PSSM om te kijken op welke posities één van de basen 18x of vaker voorkomt. Print hiervan de positie, de base en het aantal keer dat de base op de positie voorkomt. De output moet er als volgt uit komen te zien:<br>
```
29 A 18
30 T 22
...
67 A 18
100 G 18
```
e) Maak een consensus sequentie van de alignment en print hiervan posities 28 t/m 48.


In [2]:
from Bio import AlignIO
from Bio.Align import AlignInfo

# a)
AlignIO.convert("clustalw.aln", "clustal", "fasta.aln", "fasta")

# b) Voor onderstaande opdrachten kun je ook verder gaan met clustalw.aln.
fasta_alignment = AlignIO.read(open("fasta.aln"), "fasta")

alignment_length = fasta_alignment.get_alignment_length()
print("Lengte van de alignment:", alignment_length)

# c)
summary_align = AlignInfo.SummaryInfo(fasta_alignment)

# pos_specific_score_matrix lijkt outdated en geeft een waarschuwing. Die mag je voor nu negeren.
from warnings import filterwarnings
filterwarnings("ignore")

pssm = summary_align.pos_specific_score_matrix()
# print(pssm)

# d)
for pos in range(alignment_length):
    for base in ["A", "C", "G", "T"]:
        if pssm[pos][base] >= 18:
            print(pos+1, base, int(pssm[pos][base]))

# e)
consensus = summary_align.dumb_consensus()
print("De consensus sequentie is: ", consensus[27:48]) #28 t/m 48


Lengte van de alignment: 117
29 A 18
30 T 22
31 G 22
32 T 22
34 A 22
37 A 19
38 A 18
42 T 22
44 A 22
45 C 22
46 A 21
47 T 22
52 T 18
57 A 19
67 A 18
100 G 18
De consensus sequentie is:  XATGTXAXXAAXXCTXACATX


<b>Opdracht 2: BioMart</b>

Dit script maakt gebruik van de package [_biomart_](https://pypi.python.org/pypi/biomart). Deze is __niet__ voor-geinstalleerd op anaconda. Je kunt *biomart* installeren met behulp van pip (een package management system om Python software packages te installeren en managen). Open de Anaconda Prompt en typ "```pip install biomart```".

In deze opdracht gaan we [Ensembl](https://www.ensembl.org) gebruiken.

Hieronder een voorbeeld hoe je van het gen met ENSG00000085840 de upstream region (het gedeelte voor het gen; de promoter) kan ophalen.

In [4]:
from biomart import BiomartServer

server = BiomartServer("http://www.ensembl.org/biomart")
human_dataset = server.datasets['hsapiens_gene_ensembl']

"""
response = human_dataset.search({
  'filters' : {'ensembl_gene_id' : 'ENSG00000085840', 'upstream_flank' : 200},
  'attributes' : ['ensembl_gene_id', 'gene_flank']
  }, header=1)
print(response.text)
"""

## Update:
## Het kan zijn dat de upstream flank soms een foutmelding geeft.
## In dat geval een paar keer de code runnen om te kijken of het een keer goed gaat.
## Blijft het fout gaan dan bij de filters de upstream flank weghalen
## en bij de attributes de gene flank weghalen.
## Bij de opdracht hoeft dan de upstream flank niet getoond te worden.


## Werkende code zonder upstream flank
response = human_dataset.search({
  'filters' : {'ensembl_gene_id' : 'ENSG00000085840'},
  'attributes' : ['ensembl_gene_id']
  }, header=1)
print(response.text)

Gene stable ID
ENSG00000085840



Een dataset beschikt over filters en attributen. Met filters kan je aangeven aan welke criteria je data moet voldoen. Veelgebruikte filters zijn bijvoorbeeld gennamen of eiwit identifiers. De volledige lijst met beschikbare filters kun je vinden door de dictionary ```<datasetnaam>.show_filters()``` te bekijken. De volledige lijst met attributen kun je bekijken met ```<datasetnaam>.show_attributes()```.

* Gebruik de volgende filters:
    1. Ensembl genen: ENSG00000123080, ENSG00000116574, ENSG00000125885, ENSG00000129355, ENSG00000183434, ENSG00000172201, ENSG00000206557, ENSG00000204086
    2. Upstream flank van 200 bp.
* En toon de volgende attributen:
    1. Ensembl gene ID
    2. Strand waarop het gen zit
    3. Startpositie van het gen
    4. Stoppositie van het gen
    5. Transcript startpositie
    6. 'Genomic coding' startpositie
    7. Upstream flank van 200 bp 

De response van de query komt terug in de vorm van tab-delimited text.

In [6]:
#human_dataset.show_filters()
#human_dataset.show_attributes()

entrylist = ["ENSG00000123080", "ENSG00000116574", "ENSG00000125885", "ENSG00000129355", 
             "ENSG00000183434", "ENSG00000172201", "ENSG00000206557", "ENSG00000204086"]

"""
response = human_dataset.search({
  'filters' : {'ensembl_gene_id' : entrylist, 'upstream_flank' : 100},
  'attributes' : ['ensembl_gene_id', 'strand', 'start_position', 'end_position', 'transcript_start', 
                  'genomic_coding_start', 'gene_flank']
  },
  header=1)
print(response.text)
"""

# Zonder upstream flank
response = human_dataset.search({
  'filters' : {'ensembl_gene_id' : entrylist},
  'attributes' : ['ensembl_gene_id', 'strand', 'start_position', 'end_position', 'transcript_start', 
                  'genomic_coding_start']
  },
  header=1)
print(response.text)


Gene stable ID	Strand	Gene start (bp)	Gene end (bp)	Transcript start (bp)	Genomic coding start
ENSG00000116574	1	228735479	228746664	228735479	228735743
ENSG00000116574	1	228735479	228746664	228735479	228737673
ENSG00000116574	1	228735479	228746664	228735479	228743285
ENSG00000123080	1	50960745	50974634	50960745	
ENSG00000123080	1	50960745	50974634	50960745	50970369
ENSG00000123080	1	50960745	50974634	50960745	50973893
ENSG00000123080	1	50960745	50974634	50968706	50970369
ENSG00000123080	1	50960745	50974634	50968706	
ENSG00000123080	1	50960745	50974634	50968706	50973893
ENSG00000123080	1	50960745	50974634	50970245	50973893
ENSG00000123080	1	50960745	50974634	50970245	50970369
ENSG00000125885	1	5950652	5998977	5950652	
ENSG00000125885	1	5950652	5998977	5950652	5952016
ENSG00000125885	1	5950652	5998977	5950652	5952424
ENSG00000125885	1	5950652	5998977	5950652	5954608
ENSG00000125885	1	5950652	5998977	5950652	5955102
ENSG00000125885	1	5950652	5998977	5950652	5957126
ENSG00000125885	1	5950